In [1]:
!pip install scikit-learn numpy pandas

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/7d/af/03d3a7d5719d00486c296ddd876e6f07a681bc4e079cb45348d2f261a748/scikit_learn-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2f/0e/3b74e8f7c908082793adafb02753477f653ccd7e189f3ba070757d2d0e65/pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/18/44/7e8d208eb59a8224fcc474415104f13be9b378be8da63f76dfde12ec2b44/scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 773.4 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3

# Import Packages

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.metrics import make_scorer, mean_absolute_error  # Import MAE scorer and metric

# Prepare training and validation data

In [2]:
use_columns = ["price","sale_yr","sale_month","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat","long",]

train_df = pd.read_csv('data/train-v3.csv', usecols=use_columns)
val_df = pd.read_csv('data/valid-v3.csv', usecols=use_columns)

# concat training set and validation set for cross validation
all_df = pd.concat([train_df, val_df])

In [3]:
y_train = all_df.price.values
all_df.drop("price", axis=1, inplace=True)
x_train = all_df.values

In [4]:
print(x_train.shape, y_train.shape)

(15128, 18) (15128,)


# Prepare loss function(root-mean-squared-error) for cross validation

In [5]:
def rmse_cv(model):
    #rmse= np.sqrt(-cross_val_score(model, x_train, y_train, scoring="neg_mean_squared_error", cv = 5))
    # Create an MAE scorer
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)  # Set greater_is_better=False because lower MAE is better
# Use cross_val_score to evaluate the model using cross-validation with MAE scoring
    #scores = cross_val_score(model, X, y, cv=cross_val, scoring=mae_scorer)
    rmse= np.sqrt(-cross_val_score(model, x_train, y_train, scoring="mae_scorer", cv = 5))
    return(rmse)

In [6]:
model = LassoCV()
model.fit(x_train, y_train)

LassoCV()

In [7]:
rmse_cv(model).mean()

NameError: name 'make_scorer' is not defined

# Read test data

In [9]:
use_columns = ["id", "sale_yr","sale_month","bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition","grade","sqft_above","sqft_basement","yr_built","yr_renovated","zipcode","lat","long",]

test_df = pd.read_csv('data/test-v3.csv', usecols=use_columns)
testID = test_df['id']
test_df.drop("id", axis=1, inplace=True)
test_df.shape

(6485, 18)

# Make prediction with pretrained model

In [10]:
y = model.predict(test_df.values)

# Generate submission

In [11]:
sub = pd.DataFrame()
sub['id'] = testID
sub['price'] = y
sub.to_csv('submission.csv', index=False)
